## Read File containing times

In [4]:
import re

In [5]:
log_file="log_time_verbose_2digits"

In [48]:
def convert_time(str_time):
    # Expect time with format 'Minutes:seconds'
    m, s = str_time.split(':')
    return float(m)*60+float(s)

In [67]:
with open(log_file) as time_file:
    line = time_file.readline()
    
    list_logged_info = ["Command being timed", "User time (seconds)", "System time (seconds)", "Percent of CPU this job got", "Elapsed (wall clock) time (h:mm:ss or m:ss)", "Average shared text size (kbytes)", "Average unshared data size (kbytes)", "Average stack size (kbytes)", "Average total size (kbytes)", "Maximum resident set size (kbytes)", "Average resident set size (kbytes)", "Major (requiring I/O) page faults", "Minor (reclaiming a frame) page faults", "Voluntary context switches", "Involuntary context switches", "Swaps", "File system inputs", "File system outputs", "Socket messages sent", "Socket messages received", "Signals delivered", "Page size (bytes)", "Exit status"]
    
    all_targets = []
    number_of_lines = 0
    nb_info = len(list_logged_info)
    previous_info = nb_info - 1 # Initialize with previous info being the last element of list_logged_info
    skipped_lines = []
    
    
    while line:
        line = line.strip()
        #print line
        try:
            key, info = line.split(": ")
            # Did we find the expected info line (i.e. are we on the next list_logged_info line)
            expected_key = list_logged_info[(previous_info+1)%nb_info]
            if key != expected_key:
                raise Exception('Found key: "%s". Expecting: "%s". Previous key: "%s\nCommand line: %s\nline #: %d"'
                                %(key, expected_key, list_logged_info[previous_info], line, number_of_lines))
            previous_info = (previous_info + 1)%nb_info
            # Save info
            if key == 'Command being timed':
                cmd = info
            elif key == 'Percent of CPU this job got':
                percent_cpu = float(info.split('%')[0])
            elif key == 'Elapsed (wall clock) time (h:mm:ss or m:ss)':
                # Safe check
                if cmd is None:
                    raise Exception('cmd is None. Line: %s' %line)
                if percent_cpu is None:
                    raise Exception('percent_cpu is None. Line: %s' %line)
                all_targets.append((cmd, convert_time(info)*percent_cpu/100.0))
                # Reset `cmd` and `percent_cpu` just in case
                cmd = None
                percent_cpu = None
            # Move to next line
            line = time_file.readline()
            number_of_lines += 1
        except ValueError as e:
            print "Skipping a line that has multiple ':' character"
            skipped_lines.append(line)
            # Find next command that corresponds to first key
            found_first_key = False
            while not found_first_key:
                line = time_file.readline()
                number_of_lines += 1
                line = line.strip()
                split_line = line.split(": ")
                if split_line[0] == list_logged_info[0]:
                    found_first_key = True
    print("Number of lines: %d" % number_of_lines)
    print("Number of skipped lines: %s" % len(skipped_lines))
    print("Number of targets: %s" % len(all_targets))

Skipping a line that has multiple ':' character
Skipping a line that has multiple ':' character
Number of lines: 190440
Number of skipped lines: 2
Number of targets: 8276


In [82]:
total_make_time = all_targets[-1][1]
print("Total time (%s): %f" % (all_targets[-1][0], total_make_time))

Total time ("make -j24"): 42432.598400


In [83]:
sum_time = 0.0
for cmd, time in all_targets[:-2]: # Skip last target that is total time
    sum_time += time
print("Sum time all targets: %f" % sum_time)

Sum time all targets: 41875.057500


In [109]:
def TargetTime(name, pattern, debug=False):
    sum_time = 0.0
    nb_targets = 0
    for cmd, time in all_targets[:-2]: # Skip last target that is total time
        if pattern.match(cmd):
            sum_time += time
            nb_targets +=1
        elif debug:
            print cmd
    print("Sum time %s targets (%d): %f (%f%%)" % (name, nb_targets, sum_time, sum_time/total_make_time*100.0))

In [110]:
TargetTime('swig', re.compile('.*swig.*bin.*swig.*python.*'))

Sum time swig targets (820): 12877.647300 (30.348477%)


In [111]:
TargetTime('igenerator', re.compile('.*igenerator.py.*'))

Sum time igenerator targets (92): 760.875700 (1.793140%)


In [112]:
TargetTime('C++ Tests', re.compile('.*test.*Test\.cxx.*'))

Sum time C++ Tests targets (1174): 9557.376700 (22.523666%)


In [115]:
TargetTime('Link static libs', re.compile('.*/usr/bin/ar.*'))

Sum time Link static libs targets (98): 5.907200 (0.013921%)


In [116]:
TargetTime('Compilation', re.compile('.*/usr/bin/c.*'))

Sum time Compilation targets (5817): 28060.282800 (66.129070%)


In [118]:
TargetTime('Compilation C', re.compile('.*/usr/bin/cc.*'))

Sum time Compilation C targets (1102): 271.259200 (0.639271%)


In [120]:
TargetTime('Compilation C++', re.compile('.*/usr/bin/c\+\+.*'))

Sum time Compilation C++ targets (4717): 27796.908900 (65.508383%)


In [117]:
TargetTime('CMake', re.compile('.*/usr/local/bin/cmake.*'))

Sum time CMake targets (1214): 41.593200 (0.098022%)


In [121]:
TargetTime('Ranlib', re.compile('.*/usr/bin/ranlib.*'))

Sum time Ranlib targets (97): 0.318300 (0.000750%)


In [129]:
# Find list commands used:
set_commands = set()
for cmd, time in all_targets[:-2]:
    cmd_split = cmd.split(" ")
    set_commands.add(cmd_split[0])
print("All commands: %s" % str(set_commands))

All commands: set(['"../../Generators/CastXML/castxml/bin/castxml', '"make', '"/usr/bin/c++', '"/home/francois.budin/devel/itk/ITK/build/Wrapping/Generators/SwigInterface/swig/bin/swig', '"/usr/local/bin/cmake', '"env', '"/usr/bin/cc', '"/home/francois.budin/devel/itk/ITK/build/bin/H5make_libsettings"', '"/home/francois.budin/.virtualenvs/itk_dev/bin/python', '"/usr/bin/ranlib', '"make"', '"/usr/bin/ar', '"../../CastXML/castxml/bin/castxml', '"/home/francois.budin/devel/itk/ITK/build/bin/H5detect"', '"./itkmkg3states'])


In [127]:
TargetTime('castxml', re.compile('.*castxml/bin/castxml.*'))

Sum time castxml targets (818): 2962.069000 (6.980645%)


In [130]:
TargetTime('make', re.compile('.*make.*'))

Sum time make targets (1222): 176.844900 (0.416767%)


In [131]:
TargetTime('env', re.compile('.*env.*'))

Sum time env targets (225): 767.853000 (1.809583%)


In [132]:
TargetTime('H5make_libsettings', re.compile('.*H5make_libsettings.*'))

Sum time H5make_libsettings targets (3): 0.147700 (0.000348%)


In [133]:
TargetTime('python', re.compile('.*bin/python.*'))

Sum time [python] targets (222): 762.090000 (1.796001%)


In [134]:
TargetTime('H5detect', re.compile('.*H5detect.*'))

Sum time H5detect targets (3): 1.512500 (0.003564%)


In [135]:
TargetTime('itkmkg3states', re.compile('.*itkmkg3states.*'))

Sum time itkmkg3states targets (3): 0.132000 (0.000311%)


In [136]:
TargetTime('Wrapping', re.compile('.*Wrapping.*'))

Sum time Wrapping targets (2526): 26993.323300 (63.614590%)


In [137]:
TargetTime('HeaderTest', re.compile('.*HeaderTest.*'))

Sum time HeaderTest targets (390): 201.099300 (0.473926%)
